In [ ]:
# Code to read CryoSat-2 SARIn phase-unwrapped data (data acquired from Ben Smith) 
# Read delta_h data and reference DEM to produce absolute heights

# Written 2025-03-07 by W. Sauthoff (sauthoff@mines.edu)

In [7]:
import os
import xarray as xr

# Define data directories dependent on home environment
if os.getenv('HOME') == '/home/jovyan':
    DATA_DIR = '/home/jovyan/data'

In [8]:
# Import Smith and others, 2017, TC method CryoSat-2 SARIn height and dh data (closed source aquired from Ben Smith)
CS2_Smith2017 = xr.open_dataset(DATA_DIR + '/altimetry/CryoSat2/CS2_SARIn_Smith2017method/mos_2010.5_2019.0.nc')

# Assign CRS
CS2_Smith2017.rio.write_crs("EPSG:3031", inplace=True)
CS2_Smith2017

<xarray.Dataset> Size: 14GB
Dimensions:      (y: 4451, x: 5451, time: 35)
Coordinates:
  * y            (y) float64 36kB -2.185e+06 -2.184e+06 ... 2.264e+06 2.265e+06
  * x            (x) float64 44kB -2.665e+06 -2.664e+06 ... 2.784e+06 2.785e+06
    spatial_ref  int64 8B 0
  * time         (time) datetime64[ns] 280B 2010-07-02T15:00:00 ... 2019-01-01
Data variables:
    mask         (y, x) float64 194MB ...
    delta_h      (time, y, x) float64 7GB ...
    data_count   (time, y, x) float64 7GB ...
Attributes:
    fileName:                mos_2010.5_2021.5.h5
    shortName:               CS2-Smith-2017
    identifier_product_DOI:  doi:10.5194/tc-11-451-2017

## Making delta_h relative to different DEM

# Convert delta_h to absolute heights

In [9]:
# Import Smith and others, 2017, Cryosphere method CryoSat-2 (CS2) SARIn DEM data 
# https://doi.org/10.5194/tc-11-451-2017
# (closed source data aquired from co-author Ben Smith)
CS2_Smith2017_DEM = xr.open_dataset(
    DATA_DIR + '/altimetry/CryoSat2/CS2_SARIn_Smith2017method/mos_z0_2016.0.nc')

# View dataset metadata
CS2_Smith2017_DEM

<xarray.Dataset> Size: 3GB
Dimensions:    (x: 10901, y: 8901)
Coordinates:
  * x          (x) float64 87kB -2.665e+06 -2.664e+06 ... 2.784e+06 2.785e+06
  * y          (y) float64 71kB -2.185e+06 -2.184e+06 ... 2.264e+06 2.265e+06
Data variables:
    FAC        (y, x) float64 776MB ...
    SMB_a      (y, x) float64 776MB ...
    cell_area  (y, x) float64 776MB ...
    z0         (y, x) float64 776MB ...

In [41]:
# Contstruct ice-surface DEM using z0, surface mass balance (SMB), and firn air content (FAC)
# The DEM (z0) is corrected for SMB_a and FAC (i.e., they are substracted from z0) 
# To get the ice-surface elevation, we add SMB_a+FAC  
CS2_Smith2017_DEM = CS2_Smith2017_DEM.assign(DEM = CS2_Smith2017_DEM['z0'] + CS2_Smith2017_DEM['SMB_a'] + CS2_Smith2017_DEM['FAC'])
CS2_Smith2017_DEM

<xarray.Dataset> Size: 4GB
Dimensions:    (x: 10901, y: 8901)
Coordinates:
  * x          (x) float64 87kB -2.665e+06 -2.664e+06 ... 2.784e+06 2.785e+06
  * y          (y) float64 71kB -2.185e+06 -2.184e+06 ... 2.264e+06 2.265e+06
Data variables:
    FAC        (y, x) float64 776MB ...
    SMB_a      (y, x) float64 776MB ...
    cell_area  (y, x) float64 776MB ...
    z0         (y, x) float64 776MB ...
    DEM        (y, x) float64 776MB nan nan nan nan nan ... nan nan nan nan nan

In [42]:
# Plot DEM to ensure things look right
CS2_Smith2017_DEM['DEM'][:,:].plot()

In [53]:
# Interpolate CS2_Smith2017_DEM onto the coordinate grid of CS2_dh['delta_h'] to make CS2_Smith2017_DEM_agg
# so the two data sets have the same resolution
CS2_Smith2017_DEM_agg = CS2_Smith2017_DEM['DEM'].interp(x=CS2_Smith2017['delta_h'].x, y=CS2_Smith2017['delta_h'].y)

In [55]:
# Print shape of the datasets to ensure latter are the same for combining
print(CS2_Smith2017_DEM.DEM.shape)
print(CS2_Smith2017_DEM_agg.shape)
print(CS2_Smith2017.delta_h.shape)

(8901, 10901)
(4451, 5451)
(45, 4451, 5451)


In [56]:
# Assign absolute heights by combing delta_h's with the 2016 DEM
CS2_Smith2017['h'] = CS2_Smith2017['delta_h'] + CS2_Smith2017_DEM_agg

In [48]:
# Delete delta_h data variable so we can replace with delta_h relative to ATL14 DEM (vs. CS2_Smith2017_DEM['DEM'])
CS2_Smith2017 = CS2_Smith2017.drop_vars('delta_h')

In [ ]:
# Assign delta heights differencing absolute heights with the ATL14 DEM to match ICESat-2 ATL15 data product
# for congruent multi-mission CryoSat-2 to ICESat-2 time series
CS2_Smith2017['delta_h'] = CS2_Smith2017['h'] - ATL14['h']

# Add a 'description' attribute to the 'delta_h' data variable
CS2_Smith2017['delta_h'].attrs['description'] = 'Height change relative to the ATL14 datum (Jan 1, 2020) surface'

CS2_Smith2017